### Finviz Data Cleaning and Transformation Pipeline

This notebook processes the raw data downloaded from Finviz, cleans it, and transforms it into an analysis-ready format.

**Workflow:**

1.  **Load Data:** Reads a raw Finviz `.parquet` file for a specific date.
2.  **Feature Engineering:** Creates new composite columns (`Info`, `MktCap AUM`).
3.  **Data Type Conversion:**
    *   Converts currency strings (e.g., `1.5B`, `250K`) into numeric values in millions.
    *   Converts percentage strings (e.g., `12.5%`) into numeric values.
    *   Converts other object columns to their proper numeric types.
4.  **Final Processing:** Sorts the data by market capitalization, sets the `Ticker` as the index, and adds a `Rank` column.
5.  **Save & Verify:** Saves the cleaned DataFrame to a new `.parquet` file and verifies the saved file.

### Setup and Configuration

**This is the only cell you need to modify.** It contains all imports, paths, and lists of columns for processing.

In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

# --- Project Path Setup ---
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

SRC_DIR = ROOT_DIR / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

# Import config and custom utils now that path is set
from config import DATE_STR, DOWNLOAD_DIR, DEST_DIR
import utils

# --- File Path Configuration ---
# Build paths using pathlib for cross-platform compatibility
SOURCE_PATH = Path(DOWNLOAD_DIR) / f'df_finviz_{DATE_STR}_stocks_etfs.parquet'
DEST_PATH = Path(DEST_DIR) / f'{DATE_STR}_df_finviz_stocks_etfs.parquet'

# --- Column Processing Configuration ---
# Define which columns need specific cleaning operations.

# Columns to combine into the 'Info' column
INFO_COLS = ["Sector", "Industry", "Single Category", "Asset Type"]

# Columns with abbreviated currency values (B, M, K) to be converted to millions
CURRENCY_COLS = [
    'Market Cap', 'AUM', 'Sales', 'Income', 'Outstanding', 'Float', 
    'Short Interest', 'Avg Volume', 'Flows 1M', 'Flows 3M', 'Flows YTD',
    'MktCap AUM' # This is the new column we create
]

# Other columns that are numeric but stored as strings (objects)
# Note: Percentage columns are detected automatically in Step 3.
OTHER_NUMERIC_COLS = [
    "No.", "P/E", "Fwd P/E", "PEG", "P/S", "P/B", "P/C", "P/FCF",
    "Book/sh", "Cash/sh", "Dividend TTM", "EPS", "EPS next Q", "Short Ratio",
    "Curr R", "Quick R", "LTDebt/Eq", "Debt/Eq", "Beta", "ATR", "RSI",
    "Employees", "Recom", "Rel Volume", "Volume", "Target Price",
    "Prev Close", "Open", "High", "Low", "Price", "Holdings"
]

# --- Notebook Setup ---
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 2500)
%load_ext autoreload
%autoreload 2

# --- Verification ---
print(f"Source file: {SOURCE_PATH}")
print(f"Destination file: {DEST_PATH}")
print(f"Processing for date: {DATE_STR}")

Source file: C:\Users\ping\Downloads\df_finviz_2025-07-02_stocks_etfs.parquet
Destination file: c:\Users\ping\Files_win10\python\py311\stocks_v0_works\data\2025-07-02_df_finviz_stocks_etfs.parquet
Processing for date: 2025-07-02


### Step 1: Load Raw Data

Load the source Parquet file into a pandas DataFrame.

In [2]:
print(f"--- Step 1: Loading data from {SOURCE_PATH.name} ---")

try:
    df = pd.read_parquet(SOURCE_PATH, engine='pyarrow')
    print("Data loaded successfully.")
    df.info()
    display(df.head(3))
except FileNotFoundError:
    print(f"ERROR: Source file not found at {SOURCE_PATH}")
    df = None  # Ensure df is None if loading fails
except Exception as e:
    print(f"An error occurred during file loading: {e}")
    df = None

--- Step 1: Loading data from df_finviz_2025-07-02_stocks_etfs.parquet ---
Data loaded successfully.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Columns: 111 entries, No. to Tags
dtypes: object(111)
memory usage: 1.3+ MB


,No.,Ticker,Company,Index,Sector,Industry,Country,Exchange,Market Cap,P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,Book/sh,Cash/sh,Dividend,Dividend TTM,Dividend Ex Date,Payout Ratio,EPS,EPS next Q,EPS This Y,EPS Next Y,EPS Past 5Y,EPS Next 5Y,Sales Past 5Y,Sales Q/Q,EPS Q/Q,EPS YoY TTM,Sales YoY TTM,Sales,Income,EPS Surprise,Revenue Surprise,Outstanding,Float,Float %,Insider Own,Insider Trans,Inst Own,Inst Trans,Short Float,Short Ratio,Short Interest,ROA,ROE,ROIC,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M,Oper M,Profit M,Perf Week,Perf Month,Perf Quart,Perf Half,Perf Year,Perf YTD,Beta,ATR,Volatility W,Volatility M,SMA20,SMA50,SMA200,50D High,50D Low,52W High,52W Low,52W Range,All-Time High,All-Time Low,RSI,Earnings,IPO Date,Optionable,Shortable,Employees,Change from Open,Gap,Recom,Avg Volume,Rel Volume,Volume,Target Price,Prev Close,Open,High,Low,Price,Change,Single Category,Asset Type,Expense,Holdings,AUM,Flows 1M,Flows% 1M,Flows 3M,Flows% 3M,Flows YTD,Flows% YTD,Return% 1Y,Return% 3Y,Return% 5Y,Tags
0,681,CF,CF Industries Holdings Inc,S&P 500,Basic Materials,Agricultural Inputs,USA,NYSE,15.20B,12.38,15.49,-,2.48,3.23,10.81,8.17,29.03,8.68,2.11%,2.00,5/15/2025,29.67%,7.58,2.27,10.21%,-16.05%,24.82%,-1.97%,5.28%,13.13%,79.20%,25.75%,0.66%,6.13B,1.34B,33.53%,7.91%,164.66M,160.77M,97.64%,0.76%,-4.48%,102.65%,-0.21%,5.09%,2.77,8.18M,9.83%,26.14%,16.73%,2.59,2.22,0.67,0.69,37.93%,32.78%,21.80%,2.68%,0.95%,17.92%,10.74%,26.62%,10.00%,0.93,2.59,1.74%,2.61%,-1.03%,5.85%,10.19%,-10.15%,27.58%,-10.15%,39.37%,67.34 - 104.45,-21.53%,4093.48%,53.17,May 07/a,8/11/2005,Yes,Yes,2800,0.62%,0.84%,2.86,2.96M,0.80,"2,376,779",88.50,92.49,93.27,94.34,92.91,93.85,1.47%,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
1,682,WMG,Warner Music Group Corp,-,Communication Services,Entertainment,USA,NASD,15.19B,33.88,21.10,7.79,2.40,26.75,23.85,19.99,1.09,1.22,2.50%,0.72,5/27/2025,83.23%,0.86,0.29,-9.50%,11.58%,10.55%,4.35%,7.51%,-0.67%,-62.27%,-14.87%,-0.91%,6.33B,446.00M,-56.53%,-2.17%,145.03M,136.63M,94.21%,73.79%,-0.01%,28.78%,1.12%,3.63%,3.08,4.96M,4.87%,89.20%,8.79%,0.66,0.64,7.95,8.03,41.46%,16.62%,7.04%,11.30%,12.24%,-6.93%,-7.31%,-4.89%,-5.97%,1.24,0.65,2.78%,2.12%,8.91%,5.95%,-5.21%,-5.69%,14.05%,-20.44%,14.05%,25.56 - 36.64,-41.97%,35.14%,69.78,May 08/b,6/3/2020,Yes,Yes,5800,4.48%,0.25%,1.89,1.61M,2.02,"3,255,735",32.27,27.83,27.90,29.26,27.90,29.15,4.74%,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
2,683,JBHT,"J.B. Hunt Transport Services, Inc",S&P 500,Industrials,Integrated Freight & Logistics,USA,NASD,15.18B,27.79,21.85,1.91,1.26,3.93,349.66,29.58,38.98,0.44,1.15%,1.74,5/9/2025,30.96%,5.51,1.32,1.45%,24.13%,3.11%,14.57%,5.69%,-0.77%,-4.32%,-12.80%,-3.82%,12.06B,561.13M,2.15%,0.77%,99.19M,79.81M,80.46%,19.54%,0.28%,75.50%,-3.65%,3.88%,2.40,3.10M,6.73%,13.97%,11.82%,0.89,0.89,0.23,0.41,10.23%,6.87%,4.65%,7.20%,11.98%,2.20%,-11.04%,-4.36%,-10.34%,1.28,3.65,2.81%,2.37%,7.26%,9.58%,-5.09%,-0.78%,24.24%,-23.64%,24.62%,122.79 - 200.40,-30.29%,11028.73%,68.03,Jul 15/a,11/22/1983,Yes,Yes,33646,1.24%,0.05%,2.07,1.29M,0.71,"918,967",153.83,151.08,151.15,153.62,150.49,153.02,1.28%,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


### Step 2: Feature Engineering - Create Composite Columns

Combine existing columns to create more meaningful features: `Info` and `MktCap AUM`.

In [3]:
if df is not None:
    print("\n--- Step 2: Engineering new features ---")
    
    # 1. Create 'Info' column by concatenating category columns.
    for col in INFO_COLS:
        if col in df.columns:
            df[col] = df[col].replace('-', '')
    df['Info'] = df[INFO_COLS].apply(lambda row: ', '.join(filter(None, row.astype(str))), axis=1)
    print("Created 'Info' column.")

    # 2. Create 'MktCap AUM' by concatenating 'Market Cap' and 'AUM'.
    # This combines stock and ETF liquidity metrics into a single string column for now.
    # It will be converted to numeric in the next step.
    df['MktCap AUM'] = df['Market Cap'].replace('-', '') + df['AUM'].replace('-', '')
    print("Created 'MktCap AUM' column.")

    # Display the new columns for verification
    display(df[['Ticker', 'Info', 'MktCap AUM']].head(3))


--- Step 2: Engineering new features ---
Created 'Info' column.
Created 'MktCap AUM' column.


,Ticker,Info,MktCap AUM
0,CF,"Basic Materials, Agricultural Inputs",15.20B
1,WMG,"Communication Services, Entertainment",15.19B
2,JBHT,"Industrials, Integrated Freight & Logistics",15.18B


### Step 3: Data Type Conversion

This multi-part step cleans and converts all string-based numeric and percentage columns into proper numeric types.

#### Part A: Convert Abbreviated Currency Columns to Millions

In [4]:
def convert_to_millions(value: str) -> float:
    """Converts a string with a T/B/M/K suffix to a numeric value in millions."""
    if pd.isna(value):
        return np.nan
    
    value_str = str(value).strip().upper()
    if not value_str:
        return np.nan

    multipliers = {'T': 1_000_000, 'B': 1_000, 'M': 1, 'K': 0.001}
    suffix = value_str[-1]
    
    if suffix in multipliers:
        number_part = value_str[:-1]
        try:
            return float(number_part) * multipliers[suffix]
        except (ValueError, TypeError):
            return np.nan
    return np.nan

if df is not None:
    print("\n--- Step 3a: Converting currency columns to millions ---")
    new_names = {}
    for col in CURRENCY_COLS:
        if col in df.columns:
            df[col] = df[col].apply(convert_to_millions)
            new_names[col] = f"{col}, M"
    
    df.rename(columns=new_names, inplace=True)
    print(f"Converted and renamed {len(new_names)} columns.")
    display(df[[name for name in new_names.values() if name in df.columns]].head(3))


--- Step 3a: Converting currency columns to millions ---
Converted and renamed 12 columns.


,"Market Cap, M","AUM, M","Sales, M","Income, M","Outstanding, M","Float, M","Short Interest, M","Avg Volume, M","Flows 1M, M","Flows 3M, M","Flows YTD, M","MktCap AUM, M"
0,15200.0,NaN,6130.0,1340.00,164.66,160.77,8.18,2.96,NaN,NaN,NaN,15200.0
1,15190.0,NaN,6330.0,446.00,145.03,136.63,4.96,1.61,NaN,NaN,NaN,15190.0
2,15180.0,NaN,12060.0,561.13,99.19,79.81,3.10,1.29,NaN,NaN,NaN,15180.0


#### Part B: Convert Percentage Columns to Numeric

In [5]:
if df is not None:
    print("\n--- Step 3b: Converting percentage columns ---")
    percent_cols = [
        col for col in df.columns if df[col].dtype == 'object' and df[col].str.endswith('%', na=False).any()
    ]

    if not percent_cols:
        print("No new percentage columns found to modify.")
    else:
        print("Processing the following percentage columns:")
        for col in percent_cols:
            df[col] = pd.to_numeric(df[col].str.replace('%', ''), errors='coerce')
            new_name = f"{col} %" if '%' not in col else col
            df.rename(columns={col: new_name}, inplace=True)
            print(f"  - Converted '{col}' to numeric and renamed to '{new_name}'")


--- Step 3b: Converting percentage columns ---
Processing the following percentage columns:
  - Converted 'Dividend' to numeric and renamed to 'Dividend %'
  - Converted 'Payout Ratio' to numeric and renamed to 'Payout Ratio %'
  - Converted 'EPS This Y' to numeric and renamed to 'EPS This Y %'
  - Converted 'EPS Next Y' to numeric and renamed to 'EPS Next Y %'
  - Converted 'EPS Past 5Y' to numeric and renamed to 'EPS Past 5Y %'
  - Converted 'EPS Next 5Y' to numeric and renamed to 'EPS Next 5Y %'
  - Converted 'Sales Past 5Y' to numeric and renamed to 'Sales Past 5Y %'
  - Converted 'Sales Q/Q' to numeric and renamed to 'Sales Q/Q %'
  - Converted 'EPS Q/Q' to numeric and renamed to 'EPS Q/Q %'
  - Converted 'EPS YoY TTM' to numeric and renamed to 'EPS YoY TTM %'
  - Converted 'Sales YoY TTM' to numeric and renamed to 'Sales YoY TTM %'
  - Converted 'EPS Surprise' to numeric and renamed to 'EPS Surprise %'
  - Converted 'Revenue Surprise' to numeric and renamed to 'Revenue Surprise 

#### Part C: Convert Other String-Based Numeric Columns

In [6]:
if df is not None:
    print("\n--- Step 3c: Converting other numeric string columns ---")
    converted_count = 0
    for col in OTHER_NUMERIC_COLS:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = pd.to_numeric(df[col].str.replace(',', '', regex=False), errors='coerce')
            converted_count += 1
            
    print(f"Converted {converted_count} additional columns to numeric type.")
    print("\nData types after all conversions:")
    df.info()


--- Step 3c: Converting other numeric string columns ---
Converted 32 additional columns to numeric type.

Data types after all conversions:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Columns: 113 entries, No. to MktCap AUM, M
dtypes: float64(94), int64(2), object(17)
memory usage: 1.3+ MB


### Step 4: Final Processing - Sort, Index, and Rank

Sort the DataFrame by the unified liquidity metric, set the `Ticker` as the index, and add a final `Rank`.

In [7]:
if df is not None:
    print("\n--- Step 4: Finalizing DataFrame ---")
    
    # 1. Sort by the primary metric in descending order
    df.sort_values(by='MktCap AUM, M', ascending=False, inplace=True, na_position='last')
    print("Sorted DataFrame by 'MktCap AUM, M'.")
    
    # 2. Add a 'Rank' column based on the new sort order
    df['Rank'] = range(1, len(df) + 1)
    print("Added 'Rank' column.")
    
    # 3. Set 'Ticker' as the index
    if 'Ticker' in df.columns:
        df.set_index('Ticker', inplace=True)
        print("Set 'Ticker' as the index.")
    
    print("\nFinal DataFrame structure:")
    display(df[['Rank', 'Info', 'MktCap AUM, M']].head())


--- Step 4: Finalizing DataFrame ---
Sorted DataFrame by 'MktCap AUM, M'.
Added 'Rank' column.
Set 'Ticker' as the index.

Final DataFrame structure:


C:\Users\ping\AppData\Local\Temp\ipykernel_13576\3337454481.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Rank'] = range(1, len(df) + 1)


,Rank,Info,"MktCap AUM, M"
Ticker,,,
NVDA,1,"Technology, Semiconductors",3836900.0
MSFT,2,"Technology, Software - Infrastructure",3650050.0
AAPL,3,"Technology, Consumer Electronics",3172970.0
AMZN,4,"Consumer Cyclical, Internet Retail",2334750.0
GOOG,5,"Communication Services, Internet Content & Inf...",2174130.0


### Step 5: Save and Verify Cleaned Data

Save the final, cleaned DataFrame to a new Parquet file and read it back to verify integrity.

In [8]:
if df is not None:
    print("\n--- Step 5: Saving and verifying data ---")
    try:
        # Ensure destination directory exists
        DEST_PATH.parent.mkdir(parents=True, exist_ok=True)
        
        # Save the file
        df.to_parquet(DEST_PATH, engine='pyarrow', compression='zstd')
        print(f"Successfully saved cleaned data to: {DEST_PATH}")

        # Verify by loading it back
        loaded_df = pd.read_parquet(DEST_PATH, engine='pyarrow')
        print("\nVerification successful. First 20 rows of the saved file:")
        display(loaded_df.head(20))
        
    except Exception as e:
        print(f"An error occurred during save or verification: {e}")


--- Step 5: Saving and verifying data ---
Successfully saved cleaned data to: c:\Users\ping\Files_win10\python\py311\stocks_v0_works\data\2025-07-02_df_finviz_stocks_etfs.parquet

Verification successful. First 20 rows of the saved file:


,No.,Company,Index,Sector,Industry,Country,Exchange,"Market Cap, M",P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,Book/sh,Cash/sh,Dividend %,Dividend TTM,Dividend Ex Date,Payout Ratio %,EPS,EPS next Q,EPS This Y %,EPS Next Y %,EPS Past 5Y %,EPS Next 5Y %,Sales Past 5Y %,Sales Q/Q %,EPS Q/Q %,EPS YoY TTM %,Sales YoY TTM %,"Sales, M","Income, M",EPS Surprise %,Revenue Surprise %,"Outstanding, M","Float, M",Float %,Insider Own %,Insider Trans %,Inst Own %,Inst Trans %,Short Float %,Short Ratio,"Short Interest, M",ROA %,ROE %,ROIC %,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M %,Oper M %,Profit M %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,Perf YTD %,Beta,ATR,Volatility W %,Volatility M %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,52W Range,All-Time High %,All-Time Low %,RSI,Earnings,IPO Date,Optionable,Shortable,Employees,Change from Open %,Gap %,Recom,"Avg Volume, M",Rel Volume,Volume,Target Price,Prev Close,Open,High,Low,Price,Change %,Single Category,Asset Type,Expense %,Holdings,"AUM, M","Flows 1M, M",Flows% 1M,"Flows 3M, M",Flows% 3M,"Flows YTD, M",Flows% YTD,Return% 1Y,Return% 3Y,Return% 5Y,Tags,Info,"MktCap AUM, M",Rank
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NVDA,1,NVIDIA Corp,"DJIA, NDX, S&P 500",Technology,Semiconductors,USA,NASD,3836900.0,50.65,27.51,1.71,25.84,45.74,71.46,53.24,3.44,2.20,0.03,0.04,6/11/2025,1.16,3.10,1.00,44.35,32.42,91.83,29.57,64.24,69.18,27.60,81.36,86.17,148510.0,76770.0,9.89,1.68,24390.00,23410.00,95.97,4.07,-0.44,66.39,0.60,0.88,0.83,206.80,75.89,115.46,81.82,3.39,2.96,0.12,0.12,70.11,58.03,51.69,1.91,14.46,42.76,14.77,27.29,17.10,2.15,4.06,2.50,2.26,6.76,18.35,21.12,-0.92,65.46,-0.92,81.54,86.62 - 158.71,-0.92,471649.98,69.74,May 28/a,1/22/1999,Yes,Yes,36000.0,2.72,-0.14,1.38,248.11000,0.69,170481522,173.58,153.30,153.08,157.60,152.97,157.25,2.58,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Semiconductors",3836900.0,1
MSFT,2,Microsoft Corporation,"DJIA, NDX, S&P 500",Technology,Software - Infrastructure,USA,NASD,3650050.0,37.95,32.40,2.61,13.52,11.34,45.84,52.62,43.30,10.71,0.67,2.41,8/21/2025,25.42,12.94,3.37,13.50,13.16,18.45,14.55,14.33,13.27,17.88,12.10,14.13,270010.0,96640.0,7.38,2.38,7430.00,7320.00,98.50,1.48,-0.12,73.61,0.68,0.70,2.20,51.17,18.46,33.61,23.24,1.37,1.36,0.29,0.33,69.07,45.23,35.79,-0.24,6.30,28.49,14.07,9.88,16.51,1.05,6.70,1.17,1.23,2.00,8.33,15.80,-1.93,38.07,-1.93,42.43,344.79 - 500.76,-1.93,616238.33,67.97,Apr 30/a,3/13/1986,Yes,Yes,228000.0,0.22,-0.42,1.31,23.25000,0.70,16299081,524.23,492.05,489.99,493.50,488.70,491.09,-0.20,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Software - Infrastructure",3650050.0,2
AAPL,3,Apple Inc,"DJIA, NDX, S&P 500",Technology,Consumer Electronics,USA,NASD,3172970.0,33.15,27.32,4.32,7.93,47.51,65.42,32.22,4.47,3.25,0.48,1.01,5/12/2025,16.11,6.41,1.42,6.21,8.45,15.41,7.67,8.51,5.08,7.68,-0.36,4.91,400370.0,97290.0,1.39,0.86,14940.00,14920.00,99.88,0.10,-1.28,63.82,-0.17,0.67,1.57,100.23,29.10,138.02,66.93,0.82,0.78,1.18,1.47,46.63,31.81,24.30,5.40,5.32,-4.82,-16.88,0.86,-15.17,1.16,4.58,2.35,2.00,5.47,4.42,-4.79,-0.99,11.92,-18.32,25.55,169.21 - 260.10,-18.32,333841.11,66.29,May 01/a,12/12/1980,Yes,Yes,164000.0,1.65,0.57,2.00,63.64000,1.07,67807413,228.41,207.82,209.00,213.34,208.14,212.44,2.22,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Consumer Electronics",3172970.0,3
AMZN,4,Amazon.com Inc,"DJIA, NDX, S&P 500",Consumer Cyclical,Internet Retail,USA,NASD,2334750.0,35.87,30.19,2.06,3.59,7.63,23.76,112.19,28.82,9.25,NaN,NaN,-,0.00,6.13,1.32,12.21,17.40,36.89,17.39,17.86,8.62,62.33,71.88,10.08,650310.0,65940.0,16.38,0.33,10610.00,9490.00,89.45,10.58,-0.32,64.43,0.39,0.65,1.23,61.84,11.23,25.24,15.02,1.05,0.84,0.44,0.49,49.16,11.15,10.14,3.74,6.42,14.44,-1.71,13.80,0.24,1.31,4.90,2.20,2.06,2.57,7.88,7.03,-1.74,33.06,-9.32,45.06,151.61 - 242.52,-9.32,335016.20,61.57,May 01/a,5